In [1]:
import numpy as np
import pandas as pd
import yaml
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns

# Import

First we open a file containing a dictionary of columns and associated data types generated in `../../optimize/`, then we read input file.

In [2]:
with open('../input/dtypes.yaml', 'r') as yamlfile:
    column_types = yaml.load(yamlfile)

read_csv_opts = {'sep': '|',
                 'quotechar': '"',
                 'compression': 'gzip',
                 'encoding': 'utf-8',
                 'dtype': column_types,
                 'parse_dates': ['MissionDate'],
                 'infer_datetime_format': True}

df = pd.read_csv('../input/ice-air.csv.gz', **read_csv_opts)

df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1763020 entries, 0 to 1763019
Data columns (total 43 columns):
Status                  category
Sex                     category
Convictions             category
GangMember              category
ClassLvl                float32
Age                     float32
MissionDate             datetime64[ns]
MissionNumber           uint32
PULOC                   category
DropLoc                 category
StrikeFromList          float32
ReasonStruck            category
R-T                     category
Code                    category
CountryOfCitizenship    category
Juvenile                bool
MissionWeek             uint8
MissionQuarter          uint8
MissionYear             uint16
MissionMonth            uint8
Criminality             category
FamilyUnitFlag          float32
UnaccompaniedFlag       float32
AlienMasterID           uint32
MissionID               uint16
air_AirportID           float32
air_AirportName         category
air_City         

# Global variables

In [3]:
pre_drop = len(df)
df = df.drop_duplicates()
post_drop = len(df)
assert pre_drop == post_drop

In [4]:
number_of_records = len(df)
print(f'Number of records: {number_of_records}')

Number of records: 1763020


No missing values in ID fields. IDs repeat.

In [5]:
unique_MissionID = len(set(df['MissionID']))
assert sum(df['MissionID'].isnull()) == 0
print(f"Unique MissionID values: {unique_MissionID}")

Unique MissionID values: 14973


In [6]:
unique_MissionNumber = len(set(df['MissionNumber']))
assert sum(df['MissionNumber'].isnull()) == 0
print(f"Unique MissionNumber values: {unique_MissionNumber}")

Unique MissionNumber values: 14973


One to one relationship of MissionID to MissionNumber. These fields seem to be equivalent.

In [7]:
assert sum(df.groupby(['MissionID', 'MissionNumber'])['MissionNumber'].nunique() > 1) == 0

In [8]:
assert sum(df.groupby(['MissionNumber', 'MissionID'])['MissionID'].nunique() > 1) == 0

In [9]:
unique_AlienMasterID = len(set(df['AlienMasterID']))
assert sum(df['AlienMasterID'].isnull()) == 0
print(f"Unique AlienMasterID values: {unique_AlienMasterID}")

Unique AlienMasterID values: 1733555


In [10]:
earliest_record = df['MissionDate'].min()
print(f'Earliest record: {earliest_record}')

Earliest record: 2010-10-01 00:00:00


In [11]:
latest_record = df['MissionDate'].max()
print(f'Earliest record: {latest_record}')

Earliest record: 2018-12-05 00:00:00


In [12]:
number_of_pickup_airports = len(set(df['PULOC']))
number_of_dropoff_airports = len(set(df['DropLoc']))
print(f'Number of pickup airports: {number_of_pickup_airports}')
print(f'Number of dropoff airports: {number_of_dropoff_airports}')

Number of pickup airports: 84
Number of dropoff airports: 207


In [55]:
year_counts = []
for year in df['MissionYear'].unique():
    puloc_count = len(set(df.loc[df['MissionYear'] == year, 'PULOC']))
    droploc_count = len(set(df.loc[df['MissionYear'] == year, 'DropLoc']))
    year_counts.append((year, puloc_count, droploc_count))

In [92]:
expansion = pd.DataFrame(year_counts, columns=['FY', 'Unique pickup aiports', 'Unique dropoff aiports']).set_index('FY')

In [94]:
expansion.to_csv('../output/iceair-expansion.csv')

# Most common pickup, dropoff airports

In [15]:
df['air_AirportName'].value_counts().head(10)

Alexandria International Airport                        239911
Brownsville South Padre Island International Airport    204895
Phoenix-Mesa-Gateway Airport                            196150
Valley International Airport                            154138
El Paso International Airport                           136127
San Antonio International Airport                       107434
Columbus Metropolitan Airport                            80797
Harrisburg International Airport                         76348
Laredo International Airport                             56409
Miami International Airport                              55588
Name: air_AirportName, dtype: int64

In [16]:
df['air2_AirportName'].value_counts().head(10)

La Aurora Airport                                       327010
Ramon Villeda Morales International Airport             202955
Alexandria International Airport                        169316
El Salvador International Airport                       157773
Valley International Airport                            148700
Phoenix-Mesa-Gateway Airport                             93892
El Paso International Airport                            87408
Licenciado Benito Juarez International Airport           85584
Brownsville South Padre Island International Airport     66444
San Diego International Airport                          62943
Name: air2_AirportName, dtype: int64

In [17]:
pickup_countries = set(df['air_Country'])
print(f'Number of pickup countries: {len(pickup_countries)}')

Number of pickup countries: 11


In [18]:
dropoff_countries = set(df['air2_Country'])
print(f'Number of dropoff countries: {len(dropoff_countries)}')

Number of dropoff countries: 120


In [19]:
all_countries = pickup_countries.union(dropoff_countries)
print(f'Total countries in ICE Air network (including US): {len(dropoff_countries)}')

Total countries in ICE Air network (including US): 120


Top nationalities:

In [20]:
df.drop_duplicates(subset='AlienMasterID')['CountryOfCitizenship'].value_counts().head(15)

MEXICO                568039
GUATEMALA             421411
HONDURAS              304685
EL SALVADOR           256834
DOMINICAN REPUBLIC     31704
ECUADOR                25688
HAITI                  23308
NICARAGUA              18679
COLOMBIA               14520
JAMAICA                12800
INDIA                   8952
CHINA                   4459
CUBA                    3658
BANGLADESH              2790
SOMALIA                 2659
Name: CountryOfCitizenship, dtype: int64

# Dictionary of IATA codes to airport names

Move to clean, elsewhere?

In [21]:
pickup_names = df[['PULOC', 'air_AirportName']].drop_duplicates()
pickup_names.set_index('PULOC', inplace=True)
dropoff_names = df[['DropLoc', 'air2_AirportName']].drop_duplicates()
dropoff_names.set_index('DropLoc', inplace=True)

pickup_dict = pickup_names.to_dict()['air_AirportName']
dropoff_dict = dropoff_names.to_dict()['air2_AirportName']

airport_dict = {**pickup_dict, **dropoff_dict}

with open('../output/airport_dict.yaml', 'w') as outfile:
        yaml.dump(airport_dict, outfile, default_flow_style=False, allow_unicode=True)

# Finding repeat `AlienMasterID` values

In [22]:
count = df.groupby(['AlienMasterID']).size().reset_index(name='count')

count.set_index('AlienMasterID', inplace=True)

df.set_index('AlienMasterID', inplace=True)

df = df.join(count, how='left')

more_than_one = df['count'] > 1

multiple_alienID = df[more_than_one]

multiple_alienID.to_csv('../output/multiple_alienID.csv.gz', compression='gzip', sep='|')

In [74]:
df.reset_index(inplace=True)

# Most common, most passenger flight paths

In [101]:
most_common_flight_path = df.groupby(['PULOC','DropLoc'])['MissionID'].nunique().idxmax()
most_common_flight_path_count =  df.groupby(['PULOC','DropLoc'])['MissionID'].nunique()[most_common_flight_path]
most_passenger_flight_path = df.groupby(['PULOC','DropLoc'])['AlienMasterID'].nunique().idxmax()
most_passenger_flight_path_count = df.groupby(['PULOC','DropLoc'])['AlienMasterID'].nunique()[most_passenger_flight_path]

In [102]:
print(f'Most frequent flight path: {most_common_flight_path}, {most_common_flight_path_count} missions')
print(f'Flight path with most passengers: {most_passenger_flight_path}, {most_passenger_flight_path_count} passengers')

Most frequent flight path: ('KOAK', 'KSAN'), 919 missions
Flight path with most passengers: ('KBRO', 'MGGT'), 91714 passengers


In [103]:
df.groupby(['PULOC','DropLoc'])['MissionNumber'].nunique()[most_passenger_flight_path]

917